In [ ]:
import torch
import json
repo_path = 'datasets/data_repo/data_repo_duplicate_removed.json'

with open(repo_path, 'r') as f:
    data_repo = json.load(f)

In [2]:
def load_sims(path):
    sims = torch.load(path, map_location=torch.device('cpu'))
    return sims

def get_sims(i, data, cato_size=3, num_idx=50000):
    sims = data[:, i*cato_size:(i+1)*cato_size]
    
    sims = torch.mean(sims, dim=1)
    # sims = data[:, i]
    _, idx = torch.topk(sims, num_idx)
    idx = idx.cpu().tolist()
    # print(idx[:20])
    return idx

In [3]:
def get_index(sims, num, type_num=23, cato_size=3):
    idx_return = []
    num_cur = num // type_num
    while len(idx_return) < num:
        for i in range(type_num):
            idx = get_sims(i, sims, cato_size=cato_size, num_idx=num_cur)
            idx_return.extend(idx[:num_cur])
        idx_return = list(set(idx_return))
        if len(idx_return) > num:
            # print("idx stop:", num_cur)
            idx_return = idx_return[:num]
            break
        else:
            num_cur += max((num - len(idx_return)) // type_num, 1)
            idx_return = []
    idx_return = idx_return[:num]
    return idx_return


In [4]:
def get_idx_total(sims_total, type_num=23, cato_size=3, num_sims=13534):
    from tqdm import tqdm
    idx_mean_total = []
    idx_iso_total = []
    for sims in tqdm(sims_total):
        sims_mean = torch.mean(sims, dim=-1)
        _, idx = torch.topk(sims_mean, num_sims)
        idx_mean_total.append(idx.cpu().tolist())
        idx_iso = get_index(sims, num_sims, type_num=type_num, cato_size=cato_size)
        idx_iso_total.append(idx_iso)
    return idx_mean_total, idx_iso_total

def get_mixed_idx(sims_base_total, sims_diff_total, num_sims=13534, type_num=46, cato_size=3):
    from tqdm import tqdm
    mixed_idx = []
    for sims_baes, sims_diff in tqdm(zip(sims_base_total, sims_diff_total)):
        sims_total = torch.cat([sims_baes, sims_diff], dim=-1)
        idx_mix = get_index(sims_total, num_sims, type_num=type_num, cato_size=cato_size)
        mixed_idx.append(idx_mix)
    return mixed_idx

        
    

In [ ]:
name_list = []
def save_file(idx, start, path):
    data_sampled = []
    import random
    from tqdm import tqdm
    for id in idx:
        data_sampled.append(data_repo[id])
    random.seed(42)
    random.shuffle(data_sampled)
    save_path = path.format(start)

    with open(save_path, 'w') as f:
        json.dump(data_sampled, f, indent=4, ensure_ascii=False)
    print(save_path.split('/')[-1])
    name_list.append(save_path.split('/')[-1])

path_format = 'data_selections/tydiqa_{}_cluster_llama_1.json'
j = 15
for idx in idx_mean_total_diff:
    save_file(idx, j, path_format)
    j += 1

tydiqa_15_cluster_llama_1.json
